# **Homework 1 - task 28**
The main goal of this homework is to transform existing evaluation datasets into a format suitable for evaluating the linguistic skills of Large Language Models (LLMs) by reframing tasks as multi-choice Question Answering (QA) tasks, providing effective prompts, and generating distractors where necessary, all formatted in JSON Lines standard for submission.

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

# **1. Data Loading:**


First we need to download the datasets from evalita to our python eviroment



In [2]:
##### Data Reader -- English/French/Italian
!git clone https://github.com/shammur/SemEval2022Task3.git
%cd ./SemEval2022Task3/
#data/train/train_subtask-2/
!ls

c:\Users\35193\Desktop\Sapienza\1 year\Second Semester\Multilingual Natural Language Processing\Homework 1\GitHub\NLP_Sapienza\Task_28\SemEval2022Task3


Cloning into 'SemEval2022Task3'...
'ls' is not recognized as an internal or external command,
operable program or batch file.


# Sub Task 1 

**binary classification sub-task**, which consists in predicting the acceptability label assigned to each sentence of the test set

## 1.1 visualize initial data

**dev.xml** is the training (development) dataset.


**test_gold.xml** is the test dataset.

In [4]:
def visualize_original_data(file_path):
    try:
        with open(file_path, 'r') as file:
            for i in range(5):  
                line = file.readline()
                print(line.rstrip())
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Check test data
file_path = r'C:\Users\35193\Desktop\Sapienza\1 year\Second Semester\Multilingual Natural Language Processing\Homework 1\GitHub\NLP_Sapienza\Task_28\SemEval2022Task3\data\test\official_test_set_with_labels\subtask-1\It-Subtask1-labels.tsv'
visualize_original_data(file_path)


ID	Construction	Sentence	Labels
it_0	prefer	Non amo i campi , preferisco i condomini .	1
it_1	particular	Posso capire le emozioni , e in particolare la tristezza .	1
it_2	generally	Apprezzo i giochi da tavolo , e più in generale il calcio .	0
it_3	generally	Amo i film , e più in generale i western .	0


In [6]:
# Check info in train data
file_path = r'C:\Users\35193\Desktop\Sapienza\1 year\Second Semester\Multilingual Natural Language Processing\Homework 1\GitHub\NLP_Sapienza\Task_28\SemEval2022Task3\data\train\train_subtask-1\it\It-Subtask1-fold_0.tsv'
visualize_original_data(file_path)

ID	Sentence	Labels
5491	Amo i cartoni animati , ma non i sussidiari .	1
5084	Apprezzo il vino , ma non il Chianti .	1
1677	Amo i manuali , un tipo interessante di dipinto .	0
1959	Amo gli arbusti , ed anche le querce .	1


In [8]:
import os

# Change the working directory
os.chdir(r'c:\Users\35193\Desktop\Sapienza\1 year\Second Semester\Multilingual Natural Language Processing\Homework 1\GitHub\NLP_Sapienza\Task_28')

# Verify the current working directory
print("Current working directory:", os.getcwd())



Current working directory: c:\Users\35193\Desktop\Sapienza\1 year\Second Semester\Multilingual Natural Language Processing\Homework 1\GitHub\NLP_Sapienza\Task_28


In [11]:
# join the fold training data to a single file

def join_tsv_files(file_paths, output_file):
    with open(output_file, 'w') as outfile:
        for file_path in file_paths:
            with open(file_path, 'r') as infile:
                for line in infile:
                    outfile.write(line)

# Example usage
file1 = r'SemEval2022Task3\data\train\train_subtask-1\it\It-Subtask1-fold_0.tsv'
file2 = r'SemEval2022Task3\data\train\train_subtask-1\it\It-Subtask1-fold_1.tsv'
file3 = r'SemEval2022Task3\data\train\train_subtask-1\it\It-Subtask1-fold_2.tsv'
output_file = 'It-Subtask1-train.tsv'

join_tsv_files([file1, file2, file3], output_file)


In [13]:
# Check info in train data
file_path = r'It-Subtask1-train.tsv'
visualize_original_data(file_path)

ID	Sentence	Labels
5491	Amo i cartoni animati , ma non i sussidiari .	1
5084	Apprezzo il vino , ma non il Chianti .	1
1677	Amo i manuali , un tipo interessante di dipinto .	0
1959	Amo gli arbusti , ed anche le querce .	1


## 1.2 Data Reframing:
  First we need to change the type of the file from xml to json. Then we want to convert that json into the following format:
```JSON
{
    "id":       int,
    "text":     str,
    "choices":  list[str],
    "label":    int
}
```

In [14]:
import importlib

# Check if the package is installed, and install it if it is not
def check_and_install(package):
    try:
        importlib.import_module(package)
        print(f"{package} is already installed.")
    except ImportError:
        print(f"{package} is not installed. Installing...")
        import subprocess
        subprocess.check_call(["pip", "install", package])

check_and_install('xmltodict')


xmltodict is already installed.


In [15]:
import csv
import json

def tsv_to_jsonl(input_file, output_file):
    with open(input_file, 'r', newline='') as tsvfile:
        reader = csv.DictReader(tsvfile, delimiter='\t')
        data = list(reader)

    with open(output_file, 'w') as jsonfile:
        json.dump(data, jsonfile, indent=4)

tsv_file = r'SemEval2022Task3\data\test\official_test_set_with_labels\subtask-1\It-Subtask1-labels.tsv'
jsonl_file = 'It-Subtask1-test.jsonl'
tsv_to_jsonl(tsv_file, jsonl_file)


In [16]:
tsv_file = 'It-Subtask1-train.tsv'
jsonl_file = 'It-Subtask1-train.jsonl'
tsv_to_jsonl(tsv_file, jsonl_file)

## 1.3 Visualize jsonl information

In [17]:
import json
import json

def visualize_jsonl_data(json_file):
    with open(json_file, 'r') as file:
        data = json.load(file)
        
    for entry in data[:3]:  
        print("{")
        for key, value in entry.items():
            print(f'    "{key}": "{value}",')
        print("},")

json_file = 'It-Subtask1-test.jsonl'
visualize_jsonl_data(json_file)


{
    "ID": "it_0",
    "Construction": "prefer",
    "Sentence": "Non amo i campi , preferisco i condomini .",
    "Labels": "1",
},
{
    "ID": "it_1",
    "Construction": "particular",
    "Sentence": "Posso capire le emozioni , e in particolare la tristezza .",
    "Labels": "1",
},
{
    "ID": "it_2",
    "Construction": "generally",
    "Sentence": "Apprezzo i giochi da tavolo , e più in generale il calcio .",
    "Labels": "0",
},


In [18]:
json_file = 'It-Subtask1-train.jsonl'
visualize_jsonl_data(json_file)

{
    "ID": "5491",
    "Sentence": "Amo i cartoni animati , ma non i sussidiari .",
    "Labels": "1",
},
{
    "ID": "5084",
    "Sentence": "Apprezzo il vino , ma non il Chianti .",
    "Labels": "1",
},
{
    "ID": "1677",
    "Sentence": "Amo i manuali , un tipo interessante di dipinto .",
    "Labels": "0",
},



## 1.4 Format jsonl

In order to get the json file with the desired format:

1. Rename ID to id.
2. Rename Sentence to text.
3. Add a choices key with non acceptable  or aceptable.
4. Rename Labels to label (0- non acceptable and 1 aceptable)
5. Delte Contruction.

In [20]:
import json

def rearrange_json(file_path):
    with open(file_path, "r") as file:
        data = json.load(file)
        
    rearranged_data = []
    for pair in data:
        if 'ID' in pair and 'Sentence' in pair and 'Labels' in pair:
            # Rearrange the keys
            new_pair = {
                "id": pair['ID'],  
                "text": pair['Sentence'],
                "choices": ["non acceptable", "acceptable"],
                "label": pair['Labels']
            }
            rearranged_data.append(new_pair)
    
    # Write rearranged data back to the file
    with open(file_path, "w") as file:
        json.dump(rearranged_data, file, indent=2)

# Example usage
file_path_test_gold = 'It-Subtask1-test.jsonl'
rearrange_json(file_path_test_gold)

file_path_dev = 'It-Subtask1-train.jsonl'
rearrange_json(file_path_dev)


## 1.5 Visualize final jsonl

In [21]:
json_file = 'It-Subtask1-test.jsonl'
visualize_jsonl_data(json_file)

{
    "id": "it_0",
    "text": "Non amo i campi , preferisco i condomini .",
    "choices": "['non acceptable', 'acceptable']",
    "label": "1",
},
{
    "id": "it_1",
    "text": "Posso capire le emozioni , e in particolare la tristezza .",
    "choices": "['non acceptable', 'acceptable']",
    "label": "1",
},
{
    "id": "it_2",
    "text": "Apprezzo i giochi da tavolo , e più in generale il calcio .",
    "choices": "['non acceptable', 'acceptable']",
    "label": "0",
},


In [22]:
json_file = 'It-Subtask1-train.jsonl'
visualize_jsonl_data(json_file)


{
    "id": "5491",
    "text": "Amo i cartoni animati , ma non i sussidiari .",
    "choices": "['non acceptable', 'acceptable']",
    "label": "1",
},
{
    "id": "5084",
    "text": "Apprezzo il vino , ma non il Chianti .",
    "choices": "['non acceptable', 'acceptable']",
    "label": "1",
},
{
    "id": "1677",
    "text": "Amo i manuali , un tipo interessante di dipinto .",
    "choices": "['non acceptable', 'acceptable']",
    "label": "0",
},


## **2. Promt formulation**

In order to use this dataset we need to generate three prompts that can be used to get if a text is entailed to a hypothesis and then insert them into a json file. The three prompts are:

**Template 1:**


Prompt: "Valuta se il seguente testo è accettabile o meno secondo il contesto: {{text}}"

Translation: "Evaluate whether the following text is acceptable or not according to the context: {{text}}"

**Template 2:**

Prompt: "Determina se la seguente frase è coerente con il contesto: {{text}}"

Translation: "Determine if the following sentence is coherent with the context: {{text}}"

**Template 3:**

Prompt: "Decidi se il testo fornito è congruente con la situazione descritta: {{text}}"

Translation: "Decide if the provided text is congruent with the described situation: {{text}}"


In [23]:
import json

# Function to generate a JSON Lines file with a list of prompts
def generate_json(prompts, output_file):
    data = []
    for prompt in prompts:
        data.append({"prompt": prompt})

    with open(output_file, 'w') as json_file:
        json.dump(data, json_file, indent=4)

    with open(output_file, 'r', encoding='utf-8') as json_file:
        print(json_file.read())


prompts = [
    "Valuta se il seguente testo è accettabile o meno secondo il contesto: {{text}}",
    "Determina se la seguente frase è coerente con il contesto: {{text}}",
    "Decidi se il testo fornito è congruente con la situazione descritta: {{text}}"
]

output_file = "prompts.jsonl"
generate_json(prompts, output_file)
print(f"JSON file '{output_file}' generated successfully with {len(prompts)} prompts.")



[
    {
        "prompt": "Valuta se il seguente testo \u00e8 accettabile o meno secondo il contesto: {{text}}"
    },
    {
        "prompt": "Determina se la seguente frase \u00e8 coerente con il contesto: {{text}}"
    },
    {
        "prompt": "Decidi se il testo fornito \u00e8 congruente con la situazione descritta: {{text}}"
    }
]
JSON file 'prompts.jsonl' generated successfully with 3 prompts.


# **3. Llama 2 set up**

In [24]:
# Device agnostic code
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [25]:
# Configuration
runtimeFlag = device #Run on GPU (you can't run GPTQ on cpu)
cache_dir = None # by default, don't set a cache directory. This is automatically updated if you connect Google Drive.
scaling_factor = 1.0 # allows for a max sequence length of 16384*6 = 98304! Unfortunately, requires Colab Pro and a V100 or A100 to have sufficient RAM.

## 3.1 Install

In [26]:
!pip install -q -U transformers peft accelerate optimum

!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/

Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118/


In [27]:
!pip install -q -U pdfminer.six # could maybe add pre-built wheels to speed this up.

ERROR: Invalid requirement: '#'


In [28]:
# https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

##3.2 Load Model


In [29]:
import transformers
import torch
import json
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_id = "MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7"
tokenizer_llama = AutoTokenizer.from_pretrained(model_id)
model_llama = AutoModelForSequenceClassification.from_pretrained(model_id).to(device)

c:\Users\35193\miniconda3\envs\gpukernel\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# **4. Evaluate Homework prompts**

## 4.1. Test response of Promtps

In [40]:
import json
import torch

# Load your JSON file and process each pair
with open('It-Subtask1-test.jsonl', 'r') as file:
    data = json.load(file)[:5]  # Load only the first 5 pairs
    for pair in data:
        id = pair['id']
        text = pair['text']
        choices = pair['choices']
        label_index = int(pair['label'])  # Convert label_index to an integer
        label = choices[label_index]  # Get the actual label using label_index

        # Format the prompt with text and hypothesis
        prompt_formatted = f"Valuta se il seguente testo è accettabile o meno secondo il contesto: {text}"

        # Assuming you have tokenizer_llama and model_llama defined elsewhere
        input = tokenizer_llama(prompt_formatted, return_tensors="pt").to(device)
        output = model_llama(**input)
        logits = output.logits[0].detach().cpu().numpy()

        # Obtain the predicted label directly from logits
        probabilities = torch.softmax(torch.tensor(logits), -1).detach().cpu().numpy()
        print(f"Probabilities: {probabilities}")
        prediction_index = 1 if probabilities[1] > probabilities[0] else 0
        prediction_label = choices[prediction_index]
        prediction_confidence = probabilities[prediction_index] * 100
        prediction = {"label": prediction_label, "confidence": round(prediction_confidence, 1)}

        # Compare prediction with actual label
        is_correct = prediction_label == label

        # Visualize the formatted prompt and the prediction details
        print("Prompt:", prompt_formatted)
        print("Prediction:", prediction)
        print("Actual Label:", label)
        print("Correct Prediction:", is_correct)
        print()


Probabilities: [3.5034202e-04 9.9919301e-01 4.5671430e-04]
Prompt: Valuta se il seguente testo è accettabile o meno secondo il contesto: Non amo i campi , preferisco i condomini .
Prediction: {'label': 'acceptable', 'confidence': 99.9}
Actual Label: acceptable
Correct Prediction: True

Probabilities: [0.00333161 0.9951598  0.00150863]
Prompt: Valuta se il seguente testo è accettabile o meno secondo il contesto: Posso capire le emozioni , e in particolare la tristezza .
Prediction: {'label': 'acceptable', 'confidence': 99.5}
Actual Label: acceptable
Correct Prediction: True

Probabilities: [0.01640908 0.9175491  0.06604195]
Prompt: Valuta se il seguente testo è accettabile o meno secondo il contesto: Apprezzo i giochi da tavolo , e più in generale il calcio .
Prediction: {'label': 'acceptable', 'confidence': 91.8}
Actual Label: non acceptable
Correct Prediction: False

Probabilities: [0.00220563 0.9563486  0.04144568]
Prompt: Valuta se il seguente testo è accettabile o meno secondo il c

##4.2. Compare the diferent prompts

In [45]:
import json

total_correct = 0
total_pairs = 0

# Load the JSONL file and process each pair
with open('It-Subtask1-test.jsonl', 'r') as file:
    data = json.load(file) # Load only the first 5 pairs
    for pair in data:
        id = pair['id']
        text = pair['text']
        choices = pair['choices']
        label = int(pair['label'])  # Convert label_index to an integer
        #label = choices[label_index]  # Get the actual label using label_index
        # Format the prompt with actual text and hypothesis
        prompt_formatted = f"Valuta se il seguente testo \u00e8 accettabile o meno secondo il contesto: {text}"

        input = tokenizer_llama(prompt_formatted, return_tensors="pt").to(device)
        output = model_llama(**input)
        logits = output.logits[0].detach().cpu().numpy()

        # Obtain the predicted label directly from logits
        probabilities = torch.softmax(torch.tensor(logits), -1).detach().cpu().numpy()
        prediction_index = 1 if probabilities[1] > probabilities[0] else 0
        prediction_label = choices[prediction_index]

        # Compare prediction with actual label
        is_correct = prediction_label == choices[label]

        # Accumulate statistics
        total_correct += is_correct
        total_pairs += 1

# Calculate overall statistics only if at least one pair was processed
if total_pairs > 0:
    accuracy = total_correct / total_pairs * 100

    # Print statistics
    print("Overall Statistics:")
    print("Total Pairs:", total_pairs)
    print("Total Correct:", total_correct)
    print("Accuracy:", round(accuracy, 2), "%")
else:
    print("No pairs found in the JSONL file.")


Overall Statistics:
Total Pairs: 14560
Total Correct: 6852
Accuracy: 47.06 %


In [46]:
import json
import torch

def prompt_testing(prompt):
    total_correct = 0
    total_pairs = 0
    total_confidence = 0

    # Load the JSONL file and process each pair
    with open('It-Subtask1-test.jsonl', 'r') as file:
        data = json.load(file)  # Load only the first 5 pairs
        for pair in data:
            id = pair['id']
            text = pair['text']
            choices = pair['choices']
            label = int(pair['label'])  # Convert label_index to an integer
            
            # Format with text and hypothesis 
            prompt_formatted = prompt.replace('{{text}}', text)
            input = tokenizer_llama(prompt_formatted, return_tensors="pt").to(device)
            output = model_llama(**input)
            logits = output.logits[0].detach().cpu().numpy()

            # Obtain the predicted label directly from logits
            probabilities = torch.softmax(torch.tensor(logits), -1).detach().cpu().numpy()
            prediction_index = 1 if probabilities[1] > probabilities[0] else 0
            prediction_label = choices[prediction_index]

            # Compare prediction with actual label
            is_correct = prediction_label == choices[label]

            # Accumulate statistics
            total_correct += is_correct
            total_pairs += 1
            total_confidence += probabilities[prediction_index]

    # Calculate statistics 
    if total_pairs > 0:
        accuracy = total_correct / total_pairs * 100
        mean_confidence = total_confidence / total_pairs

        # Print statistics
        print("Total Pairs:", total_pairs)
        print("Total Correct:", total_correct)
        print("Accuracy:", round(accuracy, 2), "%")
        print("Mean Confidence:", round(mean_confidence, 2))
    else:
        print("No pairs found in the JSONL file.")

# Load prompts from the JSONL file and test each prompt
with open('prompts.jsonl', 'r') as prompts_file:
    prompt_data = json.load(prompts_file)
    for prompt_item in prompt_data:
        prompt = prompt_item["prompt"]
        print("\nPrompt:", prompt)
        prompt_testing(prompt)



Prompt: Valuta se il seguente testo è accettabile o meno secondo il contesto: {{text}}
Total Pairs: 14560
Total Correct: 6852
Accuracy: 47.06 %
Mean Confidence: 0.93

Prompt: Determina se la seguente frase è coerente con il contesto: {{text}}
Total Pairs: 14560
Total Correct: 6852
Accuracy: 47.06 %
Mean Confidence: 0.97

Prompt: Decidi se il testo fornito è congruente con la situazione descritta: {{text}}
Total Pairs: 14560
Total Correct: 6848
Accuracy: 47.03 %
Mean Confidence: 0.79
